Compute standard deviation and means by streaming in training batches.

Adapted from https://stackoverflow.com/a/5543790/732596

In [70]:
import xarray as xr
from pathlib import Path
import pandas as pd
import numpy as np

In [71]:
BASE_PATH = Path("/mnt/storage_ssd_4tb/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/prepared_ML_training_data/v15/train")
DATA_SOURCE_NAME = "nwp"

In [72]:
def compute_accumulators(data_array: xr.DataArray) -> pd.DataFrame:
    dims_to_aggregate_over = set(data_array.dims) - set(['channels_index'])
    data_array = data_array.astype(np.float64)  # Minimise numerical instability.
    _count = data_array.count(dim=dims_to_aggregate_over).to_pandas()
    _sum = data_array.sum(dim=dims_to_aggregate_over).to_pandas()
    _sum_of_squares = (data_array ** 2).sum(dim=dims_to_aggregate_over).to_pandas()
    return pd.Series({
        'count': _count,
        'sum': _sum.astype(np.float128),
        'sum_of_squares': _sum_of_squares.astype(np.float128)
    })

In [73]:
def compute_std(accumulators: pd.DataFrame):
    return np.sqrt(
        (accumulators['count'] * accumulators['sum_of_squares'] - accumulators['sum'] * accumulators['sum'])
        / 
        (accumulators['count'] * (accumulators['count'] - 1))
    )

In [74]:
def compute_mean(accumulators: pd.DataFrame):
    return accumulators['sum'] / accumulators['count']

In [75]:
def load_and_check_batch(filename: Path) -> pd.DataFrame:
    """Loads a batch NetCDF file. Computes stats. Returns pd.Series mapping stat name to stat value."""
    dataset = xr.load_dataset(filename, mode="r")
    data_array = dataset['data']
    
    # Validation checks:
    msg = ""
    if not np.isfinite(data_array).all():
        msg += "NOT FINITE "
    #if (data_array < 0).any():
    #    msg += f"NEGATIVE!  min={data_array.min().values} "
    #if (data_array > 1023).any():
    #    msg += f"ABOVE 1,023!  max={data_array.max().values} "
    if msg:
        print("\n", filename.stem, msg, "\n", flush=True)
    
    # Compute accumulators for standard deviation and mean:
    return compute_accumulators(data_array)

In [76]:
def run_on_all_files():
    filenames = (BASE_PATH / DATA_SOURCE_NAME).glob("*.nc")
    filenames = np.sort(list(filenames))
    n = len(filenames)
    print(n, "filenames found")
    accumulators = None
    for i, filename in enumerate(filenames):
        print(f"{i+1:5,d}/{n:5,d}: {filename}\r", flush=True, end="")
        accumulators_for_filename = load_and_check_batch(filename)
        if accumulators is None:
            accumulators = accumulators_for_filename
        else:
            accumulators += accumulators_for_filename
            
    return accumulators

accumulators = run_on_all_files()

4000 filenames found


In [77]:
accumulators

count                         524288000
sum                  71243672301.444901
sum_of_squares    20803280670966.800781
dtype: object

In [81]:
filename = BASE_PATH / DATA_SOURCE_NAME / "000000.nc"
dataset = xr.load_dataset(filename, mode="r")
dataset

<xarray.Dataset>
Dimensions:      (example: 32, y_index: 64, x_index: 64)
Coordinates:
    band         int64 1
  * example      (example) int64 0 1 2 3 4 5 6 7 8 ... 24 25 26 27 28 29 30 31
  * x_index      (x_index) int64 0 1 2 3 4 5 6 7 8 ... 56 57 58 59 60 61 62 63
  * y_index      (y_index) int64 0 1 2 3 4 5 6 7 8 ... 56 57 58 59 60 61 62 63
Data variables:
    data         (example, y_index, x_index) float32 0.0 0.0 ... 3.089 3.0
    spatial_ref  int64 0
    x            (example, x_index) float64 1.588e+05 1.608e+05 ... 5.158e+05
    y            (example, y_index) float64 7.786e+05 7.766e+05 ... 3.266e+05

In [17]:
accumulators["channel_name"] = dataset.channels[0]
accumulators = accumulators.reset_index().set_index("channel_name").sort_values("channels_index")
accumulators

,channels_index,count,sum,sum_of_squares
channel_name,,,,
IR_016,0,2285568000,6.665087e+11,2.619892e+14
IR_039,1,2285568000,1.962855e+12,1.707008e+15
IR_087,2,2285568000,1.687458e+12,1.267026e+15
IR_097,3,2285568000,1.766952e+12,1.383207e+15
IR_108,4,2285568000,1.388555e+12,8.993605e+14
IR_120,5,2285568000,1.967124e+12,1.717936e+15
IR_134,6,2285568000,2.114260e+12,1.980686e+15
VIS006,7,2285568000,5.211583e+11,1.704251e+14
VIS008,8,2285568000,5.886785e+11,2.045396e+14


In [78]:
accumulators.to_csv(f"{DATA_SOURCE_NAME}_accumulators.csv")

In [79]:
mean = compute_mean(accumulators)
#mean.to_csv(f"{DATA_SOURCE_NAME}_mean.csv")
#mean.to_dict()
mean

135.88652096070271

In [80]:
std = compute_std(accumulators)
#std.to_csv(f"{DATA_SOURCE_NAME}_std.csv")
#std.to_dict()
std

145.65013699767726867

In [84]:
dataset['data'].std()

<xarray.DataArray 'data' ()>
array(143.25692749)
Coordinates:
    band     int64 1